In [1219]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
import pickle
from shapely import LineString,MultiLineString
from shapely import geometry, ops
import numpy as np

In [1220]:
iris_data=pd.read_excel(r"Y:\REFERENTIEL DATA\RP 2022 FRANCE\GEODATA - IRIS\\Geodata_IRIS_essentiels.xlsx")
contour_iris=gpd.read_file(r'Y:\REFERENTIEL DATA\RP 2022 FRANCE\CARTOGRAPHIE\Cartographie_France_IRIS_2019\CONTOURS-IRIS_2-1_SHP_LAMB93_FXX-2019\CONTOURS-IRIS.shp').to_crs("3857")

In [1285]:
highways=pd.read_pickle(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm\paris\raw data\\ways.pkl").to_crs("3857")
batiments=pd.read_pickle(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm\paris\processed data\\buildings_est_pop.pkl").to_crs("3857")
shops=pd.read_pickle(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm\paris\raw data\\shops.pkl").to_crs("3857")
gsa_comp=pd.read_pickle(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm\paris\processed data\\gsa_comp.pkl").to_crs("3857")
gss_comp=pd.read_pickle(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm\paris\processed data\\gss_comp.pkl").to_crs("3857")

In [1222]:
loisire=pd.read_pickle(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm\paris\raw data\\leisure.pkl").to_crs("3857")
tourism=pd.read_pickle(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm\paris\raw data\\tourism.pkl").to_crs("3857")
public_transport=pd.read_pickle(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm\paris\raw data\\public_transport.pkl").to_crs("3857")
amenity=pd.read_pickle(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm\paris\raw data\\amenity.pkl").to_crs("3857")

In [1223]:
indicateurs=pd.read_excel(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\\Données_OSM_métie1.xlsx",sheet_name='shops')
indicateurs=indicateurs[indicateurs['à garder']==1]
indicateurs=indicateurs.set_index("shop")

In [1224]:
indicateurs_amenity=pd.read_excel(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\\Données_OSM_métie1.xlsx",sheet_name='amenity')
indicateurs_amenity=indicateurs_amenity[indicateurs_amenity['à garder']==1]
indicateurs_amenity=indicateurs_amenity.set_index('amenity')

In [1225]:
indicateurs_loisire=pd.read_excel(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\\Données_OSM_métie1.xlsx",sheet_name='leisure')
indicateurs_loisire=indicateurs_loisire[indicateurs_loisire['à garder']==1]
indicateurs_loisire=indicateurs_loisire.set_index('loisire')

In [1226]:
columns_socio=['C17_MEN',
       'C17_MEN_CS1', 'C17_MEN_CS2', 'C17_MEN_CS3', 'C17_MEN_CS4',
       'C17_MEN_CS5', 'C17_MEN_CS6', 'C17_MEN_CS7', 'C17_MEN_CS8', 'P17_POP',
       'P17_POP0002', 'P17_POP0305', 'P17_POP0610', 'P17_POP1117',
       'P17_POP1824', 'P17_POP2539', 'P17_POP4054', 'P17_POP5564',
       'P17_POP6579', 'P17_POP80P', 'P17_POP15P']

In [1334]:
max_distance=25
def f(x):

    if x['leisure'] in indicateurs_loisire.index:
        return indicateurs_loisire.loc[x['leisure']]['attracteur'] 
    else :
        return "Attracteur pour tous -loisire"
def g(x):
    if x['shop'] in indicateurs.index:
        return indicateurs.loc[x['shop']]['secteurs'] 
    else :
        return "Autre"
batiments_affectes=set()
tourism_affectes=set()
loisire_affectes=set()

public_transport_affectes=set()
shop_affectes=set()
amenity_affectes=set()
gsa_affectes=set()
gss_affectes=set()

def get_KPI(row):
    
    if "Avenue" in row['name'] or "Quai" in row['name'] or "Boulevard" in row['name']:
        max_distance=75
    else:max_distance=25
        
    socio_demo=batiments[batiments.intersects(row['geometry'].buffer(max_distance))][columns_socio]
    #sauvegarder les batimenets affectés à un segement
    batiments_affectes.update(list(socio_demo.index))
   
    socio_demo_index=list(socio_demo.index)
    #agregation
    socio_demo=socio_demo.sum()
    
    
    #tourism
    tourism_seg=tourism[tourism.intersects(row['geometry'].buffer(max_distance))]
    #sauvegarder les "tourisme" affectés à un segement
    tourism_affectes.update(list(tourism_seg.index))
    
    #loisir
    loisire_seg=loisire[loisire.intersects(row['geometry'].buffer(max_distance))]
    loisire_seg["attracteur"]=loisire_seg.apply(lambda x:f(x),axis=1)
    #sauvegarder les "loisire" affectés à un segement
    loisire_affectes.update(list(loisire_seg.index))
    
 
    #transport 
    public_transport_seg=public_transport[public_transport.intersects(row['geometry'].buffer(max_distance))]
    metro=public_transport_seg[~public_transport_seg['subway'].isna()].drop_duplicates('name')
    tram=public_transport_seg[public_transport_seg['railway']=="tram_stop"].drop_duplicates('name')
    reste_transport=len(public_transport_seg)-len(metro)-len(tram)
    #sauvegarder les "transport" affectés à un segement
    public_transport_affectes.update(list(public_transport_seg.index))
    
    #info shops
    shop_seg=shops[shops.intersects(row['geometry'].buffer(max_distance))]
    #sauvegarder les "shops" affectés à un segement
    shop_affectes.update(list(shop_seg.index))

    shop_seg=shop_seg[shop_seg['shop'].isin(indicateurs.index)]
    shop_seg['secteur']=shop_seg.apply(lambda x:g(x),axis=1)


    #### gsa_comp_info  
    gsa_comp_info=gsa_comp[gsa_comp.intersects(row['geometry'].buffer(max_distance))]
    #sauvegarder les "gsa" affectés à un segement
    gsa_affectes.update(list(gsa_comp_info.index))
    
    #### gss_comp_info
    gss_comp_info=gss_comp[gss_comp.intersects(row['geometry'].buffer(max_distance))]
    gss_affectes.update(list(gss_comp_info.index))
    
    #amenity
    amenity_seg=amenity[amenity.intersects(row['geometry'].buffer(max_distance))]
    amenity_affectes.update(list(amenity_seg.index))
    amenity_seg=amenity_seg[amenity_seg['amenity'].isin(indicateurs_amenity.index)]
    amenity_seg['attracteur']=indicateurs_amenity.loc[amenity_seg['amenity'].values]['attracteur'].values
    
    
    
    output=socio_demo.to_dict()
    output["Attracteur_tourisme"]=len(tourism_seg)
    output=output|loisire_seg.groupby("attracteur").size().to_dict()
    output["metro"]=len(metro)
    output["tram"]=len(tram)
    output["transport"]=reste_transport
    output=output|shop_seg.groupby("secteur").size().to_dict()
    output=output|amenity_seg.groupby("attracteur").size().to_dict()
    output=output|row.to_dict()
    
    if "GSA" in output.keys():output['GSA']+=len(gsa_comp_info)
    else:output['GSA']=len(gsa_comp_info)
        
    
    if "Sport" in output.keys():output['Sport']+=len(gss_comp_info[gss_comp_info['Type_Mag']=="Sport"])
    else:output['Sport']=len(gss_comp_info[gss_comp_info['Type_Mag']=="Sport"])
    
    
    if "Equipement maison" in output.keys():output['Equipement maison']+=len(gss_comp_info[gss_comp_info['Type_Mag']=="Négoce"])+len(gss_comp_info[gss_comp_info['Type_Mag']=="Discount Bazar"])+len(gss_comp_info[gss_comp_info['Type_Mag']=="Electrodomestique"])+len(gss_comp_info[gss_comp_info['Type_Mag']=="GMS Bricolage"])+len(gss_comp_info[gss_comp_info['Type_Mag']=="Jardinerie"])
    else:output['Equipement maison']=len(gss_comp_info[gss_comp_info['Type_Mag']=="Négoce"])+len(gss_comp_info[gss_comp_info['Type_Mag']=="Discount Bazar"])+len(gss_comp_info[gss_comp_info['Type_Mag']=="Electrodomestique"])+len(gss_comp_info[gss_comp_info['Type_Mag']=="GMS Bricolage"])+len(gss_comp_info[gss_comp_info['Type_Mag']=="Jardinerie"])
     
    
    if "Equipement Personne" in output.keys():output['Equipement Personne']+=len(gss_comp_info[gss_comp_info['Type_Mag']=="GMS Textile Habillement"])
    else:output['Equipement Personne']=len(gss_comp_info[gss_comp_info['Type_Mag']=="GMS Textile Habillement"])
        
    if "Culture Loisir" in output.keys():output["Culture Loisir"]+=len(gss_comp_info[gss_comp_info['Type_Mag']=="Jouets"])+len(gss_comp_info[gss_comp_info['Type_Mag']=="Culture Loisirs"])
    else:output["Culture Loisir"]=len(gss_comp_info[gss_comp_info['Type_Mag']=="Jouets"])+len(gss_comp_info[gss_comp_info['Type_Mag']=="Culture Loisirs"])
    
    return output ,socio_demo_index,list(tourism_seg.index),list(loisire_seg.index),list(public_transport_seg.index),list(shop_seg.index),list(amenity_seg.index),list(gsa_comp_info.index),list(gss_comp_info.index)
    
    

In [1335]:
output=highways.apply(get_KPI,axis=1)

# Resultat

In [1336]:

result=pd.DataFrame.from_dict([output.values[i][0] for i in range(len(output))]).set_index(output.index)

socio_demo_index=pd.DataFrame({"indices":[output.values[i][1] for i in range(len(output))]}).set_index(output.index)
tourism_seg_index=pd.DataFrame({"indices":[output.values[i][2] for i in range(len(output))]}).set_index(output.index)
loisire_seg_index=pd.DataFrame({"indices":[output.values[i][3] for i in range(len(output))]}).set_index(output.index)
public_transport_seg_index=pd.DataFrame({"indices":[output.values[i][4] for i in range(len(output))]}).set_index(output.index)
shop_seg_index=pd.DataFrame({"indices":[output.values[i][5] for i in range(len(output))]}).set_index(output.index)
amenity_seg_index=pd.DataFrame({"indices":[output.values[i][6] for i in range(len(output))]}).set_index(output.index)
gsa_seg_index=pd.DataFrame({"indices":[output.values[i][7] for i in range(len(output))]}).set_index(output.index)
gss_seg_index=pd.DataFrame({"indices":[output.values[i][8] for i in range(len(output))]}).set_index(output.index)

In [1337]:
result=result.fillna(0)

## Affectation des batiments

In [1338]:
#selectionner les batiments non affectés à un segement
batiments_non_aff=batiments.loc[list(batiments.index.difference(list(batiments_affectes)))]
#centre du batiment
centre=batiments_non_aff.centroid
#l'indice (position) du segment le plus proche à un batiment
index=[highways.distance(centre.iloc[i]).argmin() for i in range(len(centre))]

batiments_non_aff['seg']=list(index)
a=batiments_non_aff.groupby("seg")[columns_socio].sum()
result.loc[highways.iloc[a.index].index,columns_socio]=result.iloc[a.index][columns_socio].values+a.values

In [1339]:
#l'indice reel du segement associé à ce batiment
batiments_non_aff['seg_index']=list(highways.iloc[batiments_non_aff.seg].index)
#rendre l'indice du batiment une colonne
batiments_non_aff=batiments_non_aff.reset_index()
batiments_non_aff['bat_index']=batiments_non_aff.apply(lambda row:(row['element_type'],row['osmid']),axis=1)
#regrouper les batiments dans une list par segement 
batiments_non_aff=batiments_non_aff[["bat_index","seg_index"]].groupby('seg_index')['bat_index'].apply(list)
socio_demo_index.loc[batiments_non_aff.index,'indices']+=pd.DataFrame(batiments_non_aff)['bat_index']

## Affectation des loisire

In [1340]:
#centroid des "loisire" non affectés 
loisire_non_aff=loisire.loc[list(loisire.index.difference(list(loisire_affectes)))]
centre=loisire_non_aff.centroid
index=[highways.distance(centre.iloc[i]).argmin() for i in range(len(centre))]

loisire_non_aff['seg']=list(index)
loisire_non_aff["attracteur"]=loisire_non_aff.apply(lambda x:f(x),axis=1)
a=loisire_non_aff.groupby(["seg","attracteur"]).size().reset_index().pivot("seg",'attracteur',0).fillna(0)
result.loc[highways.iloc[a.index].index,a.columns]=result.iloc[a.index][a.columns].values+a.values

C:\Users\Diametrix\AppData\Local\Temp\ipykernel_14836\2452846591.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  a=loisire_non_aff.groupby(["seg","attracteur"]).size().reset_index().pivot("seg",'attracteur',0).fillna(0)


In [1341]:

#l'indice reel du segement associé à ce batiment
loisire_non_aff['seg_index']=list(highways.iloc[loisire_non_aff.seg].index)
#rendre l'indice du batiment une colonne
loisire_non_aff=loisire_non_aff.reset_index()
loisire_non_aff['loisire_index']=loisire_non_aff.apply(lambda row:(row['element_type'],row['osmid']),axis=1)
#regrouper les batiments dans une list par segement 
loisire_non_aff=loisire_non_aff[["loisire_index","seg_index"]].groupby('seg_index')['loisire_index'].apply(list)
loisire_seg_index.loc[loisire_non_aff.index,'indices']+=pd.DataFrame(loisire_non_aff)['loisire_index']

## Affectation des tourismes

In [1342]:
#centroid des "tourisme" non affectés 
tourisme_non_aff=tourism.loc[list(tourism.index.difference(list(tourism_affectes)))]
centre=tourisme_non_aff.centroid
index=[highways.distance(centre.iloc[i]).argmin() for i in range(len(centre))]

tourisme_non_aff['seg']=list(index)
a=tourisme_non_aff.groupby("seg").size()
result.loc[highways.iloc[a.index].index,"Attracteur_tourisme"]=result.iloc[a.index]["Attracteur_tourisme"].values+a.values

In [1343]:

#l'indice reel du segement associé à ce batiment
tourisme_non_aff['seg_index']=list(highways.iloc[tourisme_non_aff.seg].index)
#rendre l'indice du batiment une colonne
tourisme_non_aff=tourisme_non_aff.reset_index()
tourisme_non_aff['tourisme_index']=tourisme_non_aff.apply(lambda row:(row['element_type'],row['osmid']),axis=1)
#regrouper les batiments dans une list par segement 
tourisme_non_aff=tourisme_non_aff[["tourisme_index","seg_index"]].groupby('seg_index')['tourisme_index'].apply(list)
tourism_seg_index.loc[tourisme_non_aff.index,'indices']+=pd.DataFrame(tourisme_non_aff)['tourisme_index']

## Affectation des transport publiques

In [1344]:
#centroid des "public_transport_affectes" non affectés 
public_transport_non_aff=public_transport.loc[list(public_transport.index.difference(list(public_transport_affectes)))]
centre=public_transport_non_aff.centroid
index=[highways.distance(centre.iloc[i]).argmin() for i in range(len(centre))]

public_transport_non_aff['seg']=list(index)

metro=public_transport_non_aff[~public_transport_non_aff['subway'].isna()].drop_duplicates('name').groupby("seg").size()
tram=public_transport_non_aff[public_transport_non_aff['railway']=="tram_stop"].drop_duplicates('name').groupby("seg").size()
transport=public_transport_non_aff.groupby("seg").size().sub(tram,fill_value=0).sub(metro,fill_value=0)

result.loc[highways.iloc[metro.index].index,"metro"]=result.iloc[metro.index]["metro"].values+metro.values
result.loc[highways.iloc[tram.index].index,"tram"]=result.iloc[tram.index]["tram"].values+tram.values
result.loc[highways.iloc[transport.index].index,"transport"]=result.iloc[transport.index]["transport"].values+transport.values

In [1345]:

#l'indice reel du segement associé à ce batiment
public_transport_non_aff['seg_index']=list(highways.iloc[public_transport_non_aff.seg].index)
#rendre l'indice du batiment une colonne
public_transport_non_aff=public_transport_non_aff.reset_index()
public_transport_non_aff['transport_index']=public_transport_non_aff.apply(lambda row:(row['element_type'],row['osmid']),axis=1)
#regrouper les batiments dans une list par segement 
public_transport_non_aff=public_transport_non_aff[["transport_index","seg_index"]].groupby('seg_index')['transport_index'].apply(list)
public_transport_seg_index.loc[public_transport_non_aff.index,'indices']+=pd.DataFrame(public_transport_non_aff)['transport_index']

## Affectation des Shops

In [1346]:
#centroid des "shops" non affectés 
shop_non_aff=shops.loc[list(shops.index.difference(list(shop_affectes)))]
centre=shop_non_aff.centroid
index=[highways.distance(centre.iloc[i]).argmin() for i in range(len(centre))]

shop_non_aff['seg']=list(index)
shop_non_aff=shop_non_aff[shop_non_aff['shop'].isin(indicateurs.index)]
shop_non_aff['secteur']=shop_non_aff.apply(lambda x:g(x),axis=1)

a=shop_non_aff.groupby(["seg","secteur"]).size().reset_index().pivot("seg",'secteur',0).fillna(0)
result.loc[highways.iloc[a.index].index,a.columns]=result.iloc[a.index][a.columns].values+a.values

C:\Users\Diametrix\AppData\Local\Temp\ipykernel_14836\451859671.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  a=shop_non_aff.groupby(["seg","secteur"]).size().reset_index().pivot("seg",'secteur',0).fillna(0)


In [1347]:

#l'indice reel du segement associé à ce batiment
shop_non_aff['seg_index']=list(highways.iloc[shop_non_aff.seg].index)
#rendre l'indice du batiment une colonne
shop_non_aff=shop_non_aff.reset_index()
shop_non_aff['shop_index']=shop_non_aff.apply(lambda row:(row['element_type'],row['osmid']),axis=1)
#regrouper les batiments dans une list par segement 
shop_non_aff=shop_non_aff[["shop_index","seg_index"]].groupby('seg_index')['shop_index'].apply(list)
shop_seg_index.loc[shop_non_aff.index,'indices']+=pd.DataFrame(shop_non_aff)['shop_index']

## Affectation des amenities

In [1348]:
#centroid des "amenity" non affectés 
amenity_non_aff=amenity.loc[list(amenity.index.difference(list(amenity_affectes)))]
centre=amenity_non_aff.centroid
index=[highways.distance(centre.iloc[i]).argmin() for i in range(len(centre))]

amenity_non_aff['seg']=list(index)
amenity_non_aff=amenity_non_aff[amenity_non_aff['amenity'].isin(indicateurs_amenity.index)]
amenity_non_aff['attracteur']=indicateurs_amenity.loc[amenity_non_aff['amenity'].values]['attracteur'].values
a=amenity_non_aff.groupby(['seg','attracteur']).size().reset_index().pivot("seg",'attracteur',0).fillna(0)
result.loc[highways.iloc[a.index].index,a.columns]=result.iloc[a.index][a.columns].values+a.values

C:\Users\Diametrix\AppData\Local\Temp\ipykernel_14836\1307367824.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  a=amenity_non_aff.groupby(['seg','attracteur']).size().reset_index().pivot("seg",'attracteur',0).fillna(0)


In [1349]:
#l'indice reel du segement associé à ce batiment
amenity_non_aff['seg_index']=list(highways.iloc[amenity_non_aff.seg].index)
#rendre l'indice du batiment une colonne
amenity_non_aff=amenity_non_aff.reset_index()
amenity_non_aff['amenity_index']=amenity_non_aff.apply(lambda row:(row['element_type'],row['osmid']),axis=1)
#regrouper les batiments dans une list par segement 
amenity_non_aff=amenity_non_aff[["amenity_index","seg_index"]].groupby('seg_index')['amenity_index'].apply(list)
amenity_seg_index.loc[amenity_non_aff.index,'indices']+=pd.DataFrame(amenity_non_aff)['amenity_index']

## Affectation des GSA

In [1350]:
#centroid des "gsa" non affectés 
gsa_non_aff=gsa_comp.loc[list(gsa_comp.index.difference(list(gsa_affectes)))]
centre=gsa_non_aff.centroid
index=[highways.distance(centre.iloc[i]).argmin() for i in range(len(centre))]

gsa_non_aff['seg']=list(index)
a=gsa_non_aff.groupby('seg').size()
result.loc[highways.iloc[a.index].index,"GSA"]+=a.values

In [1351]:
#l'indice reel du segement associé à ce batiment
gsa_non_aff['seg_index']=list(highways.iloc[gsa_non_aff.seg].index)
gsa_non_aff=gsa_non_aff.reset_index()
gsa_non_aff=gsa_non_aff[["index","seg_index"]].groupby('seg_index')['index'].apply(list)
gsa_seg_index.loc[gsa_non_aff.index,'indices']+=pd.DataFrame(gsa_non_aff)['index']

## Affectation des GSS

In [1352]:
#centroid des "gss" non affectés 
gss_non_aff=gss_comp.loc[list(gss_comp.index.difference(list(gss_affectes)))]
centre=gss_non_aff.centroid
index=[highways.distance(centre.iloc[i]).argmin() for i in range(len(centre))]

gss_non_aff['seg']=list(index)
#else:output['Equipement maison']=len(gss_comp_info[gss_comp_info['Type_Mag']=="Jardinerie"])
gss_non_aff["Type_Mag"]=gss_non_aff["Type_Mag"].replace("Négoce","Equipement maison")
gss_non_aff["Type_Mag"]=gss_non_aff["Type_Mag"].replace("Discount Bazar","Equipement maison")
gss_non_aff["Type_Mag"]=gss_non_aff["Type_Mag"].replace("Electrodomestique","Equipement maison")
gss_non_aff["Type_Mag"]=gss_non_aff["Type_Mag"].replace("GMS Bricolage","Equipement maison")
gss_non_aff["Type_Mag"]=gss_non_aff["Type_Mag"].replace("Jardinerie","Equipement maison")
gss_non_aff["Type_Mag"]=gss_non_aff["Type_Mag"].replace("GMS Textile Habillement","Equipement Personne")
gss_non_aff["Type_Mag"]=gss_non_aff["Type_Mag"].replace("Jouets","Culture Loisir")
gss_non_aff["Type_Mag"]=gss_non_aff["Type_Mag"].replace("Jouets","Culture Loisirs")      

a=gss_non_aff.groupby(['seg','Type_Mag']).size().reset_index().pivot("seg",'Type_Mag',0).fillna(0)  
result.loc[highways.iloc[a.index].index,a.columns]=result.iloc[a.index][a.columns].values+a.values

C:\Users\Diametrix\AppData\Local\Temp\ipykernel_14836\1294819167.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  a=gss_non_aff.groupby(['seg','Type_Mag']).size().reset_index().pivot("seg",'Type_Mag',0).fillna(0)


In [1353]:
#l'indice reel du segement associé à ce batiment
gss_non_aff['seg_index']=list(highways.iloc[gss_non_aff.seg].index)
gss_non_aff=gss_non_aff.reset_index()
gss_non_aff=gss_non_aff[["index","seg_index"]].groupby('seg_index')['index'].apply(list)
gss_seg_index.loc[gss_non_aff.index,'indices']+=pd.DataFrame(gss_non_aff)['index']

In [1354]:
data_viz_path=r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\04_données clients (source)\osm\paris\viz data\\"

In [1355]:

socio_demo_index.to_csv(data_viz_path+"socio_demo_index.csv")
tourism_seg_index.to_csv(data_viz_path+"tourism_seg_index.csv")
loisire_seg_index.to_csv(data_viz_path+"loisire_seg_index.csv")
public_transport_seg_index.to_csv(data_viz_path+"public_transport_seg_index.csv")
shop_seg_index.to_csv(data_viz_path+"shop_seg_index.csv")
amenity_seg_index.to_csv(data_viz_path+"amenity_seg_index.csv")
gsa_seg_index.to_csv(data_viz_path+"gsa_seg_index.csv")
gss_seg_index.to_csv(data_viz_path+"gss_seg_index.csv")

In [1369]:
result.to_excel(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\\Données_OSM_métie.xlsx")

In [1358]:
cols=list(result.columns)
cols.remove('geometry')
cols.remove('name')
cols.remove('highway')

In [1359]:
result_=result.copy()

In [1360]:
result=result_.copy()

In [1361]:
result=result_.copy()
for c in cols:
    result[c]=pd.cut(result[c],10,labels=False)
result[cols]+=1

In [1363]:
result['metro']=result['metro'].replace(
    {0:1,
     1:8,
     2:8.5,3:9,4:9.5,
     5:10}
)
result['tram']=result['tram'].replace({0:1,1:9,2:10})

In [1364]:
selected_columns=['Attracteur_tourisme',  'transport', 'Mobilité',
       'Service public', "pt d'intérert", 'restauration',  'GSA', 'Sport', 'Equipement maison',
       'Equipement Personne', 'Culture Loisir', 'Alimentaire', 'Medical',
       'Soin Personne', 'Soin de la personne et Médical -amenity',
       'Culture loisir -loisire', 'tous -amenity', 'Culture loisir -amenity',
       'Sport -loisire', 'Attracteur pour tous -loisire']

In [1365]:
min_score=[7,7,6,7,7,7,9,7,7,7,7,7,7,7,7,7,7,7,7,7]

In [1366]:
for i,c in enumerate(selected_columns):
    min=min_score[i]
    replacement={0:1}
    a=np.array(result[c].value_counts().index[1:])
    replacement.update(dict(zip(a.squeeze(),pd.qcut(a,10,labels=False)/9*(10-min)+min)))
    result[c]=result[c].replace(replacement)

In [1367]:
result.describe().to_excel(r"Y:\RECHERCHE ET DEV\10_Modèle d'attractivite des rues\\stat.xlsx")